In [1]:
import pandas as pd
import numpy as np
import os, sys, gzip, pyarrow.parquet

Local_google_drive = "/Users/siavash/Library/CloudStorage/GoogleDrive-siavash.jafarizadeh@gmail.com/.shortcut-targets-by-id/1BA94HYNI6NLWOcU5Ts7Nfso4uuVjrDwp/deeplearning2026"

df = pd.read_parquet('FNDDSeverything.parquet.gzip')

In [ ]:
print(pd.io.parquet.get_engine('auto')) 
print('hello')

In [ ]:
df.info()
df.describe().T

In [ ]:

data = df.select_dtypes(exclude="str")
d= data.fillna(0, inplace=True)
d.info()